Just need pandas to mung the data, will do some exploratory stuff in Tableau: https://public.tableau.com/profile/richard.muir#!/

In [1]:
import pandas as pd

In [2]:
!ls

Accidents_2015.csv                  Road-Safety-Dash-App.py
Casualties_2015.csv                 RoadSafetyData_2015.zip
Data & Chart Design Sandpit.ipynb   Vehicles_2015.csv
Initial data munging.ipynb          castualties2015_V.csv
Procfile                            requirements.txt
README.md                           runtime.txt
Road-Accident-Safety-Data-Guide.xls vehicles2015_V.csv


In [3]:
acc = pd.read_csv("Accidents_2015.csv")
cas = pd.read_csv("Casualties_2015.csv")
veh = pd.read_csv("Vehicles_2015.csv")

/Users/richardmuir/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
acc.head()

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location
0,201501BS70001,525130.0,180050.0,-0.198465,51.505538,1,3,1,1,12/01/2015,...,0,0,4,1,1,0,0,1,1,E01002825
1,201501BS70002,526530.0,178560.0,-0.178838,51.491836,1,3,1,1,12/01/2015,...,0,0,1,1,1,0,0,1,1,E01002820
2,201501BS70004,524610.0,181080.0,-0.205590,51.514910,1,3,1,1,12/01/2015,...,0,1,4,2,2,0,0,1,1,E01002833
3,201501BS70005,524420.0,181080.0,-0.208327,51.514952,1,3,1,1,13/01/2015,...,0,0,1,1,2,0,0,1,2,E01002874
4,201501BS70008,524630.0,179040.0,-0.206022,51.496572,1,2,2,1,09/01/2015,...,0,5,1,2,2,0,0,1,2,E01002814


Figure out which columns I want to keep:

In [5]:
acc.columns.tolist()

['Accident_Index',
 'Location_Easting_OSGR',
 'Location_Northing_OSGR',
 'Longitude',
 'Latitude',
 'Police_Force',
 'Accident_Severity',
 'Number_of_Vehicles',
 'Number_of_Casualties',
 'Date',
 'Day_of_Week',
 'Time',
 'Local_Authority_(District)',
 'Local_Authority_(Highway)',
 '1st_Road_Class',
 '1st_Road_Number',
 'Road_Type',
 'Speed_limit',
 'Junction_Detail',
 'Junction_Control',
 '2nd_Road_Class',
 '2nd_Road_Number',
 'Pedestrian_Crossing-Human_Control',
 'Pedestrian_Crossing-Physical_Facilities',
 'Light_Conditions',
 'Weather_Conditions',
 'Road_Surface_Conditions',
 'Special_Conditions_at_Site',
 'Carriageway_Hazards',
 'Urban_or_Rural_Area',
 'Did_Police_Officer_Attend_Scene_of_Accident',
 'LSOA_of_Accident_Location']

In [6]:
keeps = ['Longitude',
         'Latitude',
         'Accident_Index',
         'Police_Force',
         'Accident_Severity',
         'Number_of_Vehicles',
         'Number_of_Casualties',
         'Date',
         'Day_of_Week',
         'Time',
         'Local_Authority_(District)',
         'Speed_limit',
        ]
acc.drop([c for c in acc.columns if c not in keeps], axis = 1, inplace = True)
acc.head()

,Accident_Index,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Speed_limit
0,201501BS70001,-0.198465,51.505538,1,3,1,1,12/01/2015,2,18:45,12,30
1,201501BS70002,-0.178838,51.491836,1,3,1,1,12/01/2015,2,07:50,12,30
2,201501BS70004,-0.205590,51.514910,1,3,1,1,12/01/2015,2,18:08,12,30
3,201501BS70005,-0.208327,51.514952,1,3,1,1,13/01/2015,3,07:40,12,30
4,201501BS70008,-0.206022,51.496572,1,2,2,1,09/01/2015,6,07:30,12,30


The variables here are defined by codes to keep the file size down. I want to merge on the actual values of the variables to make for easier analysis. I'll write a function to do this for each dataset.

The codes for each variable are stored on a sheet in an XL workbook. The sheet names are pretty close to the variable names in the dataset. Hopefully will be able to make this work OK.

In [7]:
import xlrd
from copy import copy
wb = xlrd.open_workbook('Road-Accident-Safety-Data-Guide.xls')
wb.sheet_names()

['Introduction',
 'Export Variables',
 'Police Force',
 'Accident Severity',
 'Day of Week',
 'Local Authority (District)',
 'Local Authority (Highway)',
 '1st Road Class',
 'Road Type',
 'Junction Detail',
 'Junction Control',
 '2nd Road Class',
 'Ped Cross - Human',
 'Ped Cross - Physical',
 'Light Conditions',
 'Weather',
 'Road Surface',
 'Special Conditions at Site',
 'Carriageway Hazards',
 'Urban Rural',
 'Police Officer Attend',
 'Vehicle Type',
 'Towing and Articulation',
 'Vehicle Manoeuvre',
 'Vehicle Location',
 'Junction Location',
 'Skidding and Overturning',
 'Hit Object in Carriageway',
 'Vehicle Leaving Carriageway',
 'Hit Object Off Carriageway',
 '1st Point of Impact',
 'Was Vehicle Left Hand Drive',
 'Journey Purpose of Driver',
 'Sex of Driver',
 'Age Band',
 'Vehicle Propulsion Code',
 'Casualty Class',
 'Sex of Casualty',
 'Age of Casualty',
 'Casualty Severity',
 'Ped Location',
 'Ped Movement',
 'Car Passenger',
 'Bus Passenger',
 'Ped Road Maintenance Worker',

In [8]:
def mergeOnVars(df, ignore=[]):
    df2 = df.copy() # Don't overwrite because I want to see if it worked

    # Get the columns of the DF, ignoring any that I need to
    columns = [col for col in df.columns if col not in ignore]
    # Convert them to lower for easier comparison with the worksheet names
    columnsLower = [col.replace('_', '').lower() for col in df.columns if col not in ignore]
    
    for sheet in wb.sheet_names():
        replacementSheetName = sheet.replace(' ', '').lower() # Convert the worksheet names to lower for comparison with df
        if replacementSheetName in columnsLower:
            #Find the actual column name
            replacementColumn = columns[columnsLower.index(replacementSheetName)]
            
            print(replacementColumn) # Checking

            # Read in the variable coding from the worksheet
            replacement = pd.read_excel('Road-Accident-Safety-Data-Guide.xls',
                                          sheetname=sheet).rename(columns={'label' : replacementColumn})
            # Rename the column which is to be decoded
            df2.rename(columns={replacementColumn : '_{}'.format(replacementColumn)}, 
                       inplace = True)
            #Merge on the decoded variable, then drop the original
            df2 = pd.merge(df2,
                           replacement,
                           left_on='_{}'.format(replacementColumn),
                          right_on='code').drop(['_{}'.format(replacementColumn), 'code'],
                                                axis=1)
    return df2
            

This worked pretty well.

In [9]:
acc2 = mergeOnVars(acc)

Police_Force
Accident_Severity
Day_of_Week
Local_Authority_(District)


In [10]:
acc2.head()

,Accident_Index,Longitude,Latitude,Number_of_Vehicles,Number_of_Casualties,Date,Time,Speed_limit,Police_Force,Accident_Severity,Day_of_Week,Local_Authority_(District)
0,201501BS70001,-0.198465,51.505538,1,1,12/01/2015,18:45,30,Metropolitan Police,Slight,Monday,Kensington and Chelsea
1,201501BS70002,-0.178838,51.491836,1,1,12/01/2015,07:50,30,Metropolitan Police,Slight,Monday,Kensington and Chelsea
2,201501BS70004,-0.205590,51.514910,1,1,12/01/2015,18:08,30,Metropolitan Police,Slight,Monday,Kensington and Chelsea
3,201501BS70061,-0.193059,51.500150,2,1,09/02/2015,21:16,30,Metropolitan Police,Slight,Monday,Kensington and Chelsea
4,201501BS70074,-0.203266,51.508040,2,1,09/02/2015,11:30,30,Metropolitan Police,Slight,Monday,Kensington and Chelsea


Repeat above procedure for casualty dataset:

In [11]:
cas.head()

,Accident_Index,Vehicle_Reference,Casualty_Reference,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type,Casualty_IMD_Decile
0,201597UA71710,2,1,1,2,75,10,3,0,0,0,0,0,9,3,-1
1,201597UA71810,2,1,2,2,63,9,2,0,0,0,4,0,11,3,-1
2,201597UA71810,2,2,2,2,75,10,2,0,0,0,4,0,11,1,-1
3,201597UA71810,2,3,2,1,78,11,2,0,0,0,4,0,11,1,-1
4,201597UA71810,2,4,2,1,67,10,2,0,0,0,4,0,11,1,-1


In [12]:
cas.columns.tolist()

['Accident_Index',
 'Vehicle_Reference',
 'Casualty_Reference',
 'Casualty_Class',
 'Sex_of_Casualty',
 'Age_of_Casualty',
 'Age_Band_of_Casualty',
 'Casualty_Severity',
 'Pedestrian_Location',
 'Pedestrian_Movement',
 'Car_Passenger',
 'Bus_or_Coach_Passenger',
 'Pedestrian_Road_Maintenance_Worker',
 'Casualty_Type',
 'Casualty_Home_Area_Type',
 'Casualty_IMD_Decile']

In [13]:
keeps = [
    'Accident_Index',
     'Vehicle_Reference',
     'Casualty_Reference',
     'Casualty_Class',
     'Sex_of_Casualty',
     'Age_of_Casualty',
    'Casualty_Severity',
]
cas.drop([c for c in cas.columns if c not in keeps], axis = 1, inplace = True)
cas.head()

,Accident_Index,Vehicle_Reference,Casualty_Reference,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Casualty_Severity
0,201597UA71710,2,1,1,2,75,3
1,201597UA71810,2,1,2,2,63,2
2,201597UA71810,2,2,2,2,75,2
3,201597UA71810,2,3,2,1,78,2
4,201597UA71810,2,4,2,1,67,2


In [14]:
cas2 = mergeOnVars(cas, ['Age_of_Casualty'])
cas2.head()

Casualty_Class
Sex_of_Casualty
Casualty_Severity


,Accident_Index,Vehicle_Reference,Casualty_Reference,Age_of_Casualty,Casualty_Class,Sex_of_Casualty,Casualty_Severity
0,201597UA71710,2,1,75,Driver or rider,Female,Slight
1,201597UA71912,2,3,38,Driver or rider,Female,Slight
2,201597UC00105,2,1,20,Driver or rider,Female,Slight
3,201597UC00108,1,1,36,Driver or rider,Female,Slight
4,201597UC00205,1,1,51,Driver or rider,Female,Slight


Repeat above procedure for vehicle dataset:

In [15]:
veh.head()

,Accident_Index,Vehicle_Reference,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,...,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_IMD_Decile,Driver_Home_Area_Type,Vehicle_IMD_Decile
0,201506E098757,2,9,0,18,0,8,0,0,0,...,6,1,45,7,1794,1,11,-1,1,-1
1,201506E098766,1,9,0,9,0,8,0,0,0,...,6,2,25,5,1582,2,1,-1,-1,-1
2,201506E098766,2,9,0,18,0,8,0,0,0,...,6,1,51,8,-1,-1,-1,-1,1,-1
3,201506E098777,1,20,0,4,0,0,0,0,0,...,1,1,50,8,4462,2,1,-1,1,-1
4,201506E098780,1,9,0,15,0,1,0,0,0,...,6,1,27,6,1598,2,-1,-1,1,-1


In [16]:
veh.columns.tolist()

['Accident_Index',
 'Vehicle_Reference',
 'Vehicle_Type',
 'Towing_and_Articulation',
 'Vehicle_Manoeuvre',
 'Vehicle_Location-Restricted_Lane',
 'Junction_Location',
 'Skidding_and_Overturning',
 'Hit_Object_in_Carriageway',
 'Vehicle_Leaving_Carriageway',
 'Hit_Object_off_Carriageway',
 '1st_Point_of_Impact',
 'Was_Vehicle_Left_Hand_Drive?',
 'Journey_Purpose_of_Driver',
 'Sex_of_Driver',
 'Age_of_Driver',
 'Age_Band_of_Driver',
 'Engine_Capacity_(CC)',
 'Propulsion_Code',
 'Age_of_Vehicle',
 'Driver_IMD_Decile',
 'Driver_Home_Area_Type',
 'Vehicle_IMD_Decile']

In [17]:
keeps = [
    'Accident_Index',
 'Vehicle_Reference',
 'Vehicle_Type',
    'Skidding_and_Overturning',
 'Hit_Object_in_Carriageway',
 'Vehicle_Leaving_Carriageway',
 'Hit_Object_off_Carriageway',
    'Journey_Purpose_of_Driver',
 'Sex_of_Driver',
 'Age_of_Driver',
 'Engine_Capacity_(CC)',
 'Age_of_Vehicle',
]
veh.drop([c for c in veh.columns if c not in keeps], axis = 1, inplace = True)
veh.head()

,Accident_Index,Vehicle_Reference,Vehicle_Type,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,Hit_Object_off_Carriageway,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Engine_Capacity_(CC),Age_of_Vehicle
0,201506E098757,2,9,0,0,0,0,6,1,45,1794,11
1,201506E098766,1,9,0,0,0,0,6,2,25,1582,1
2,201506E098766,2,9,0,0,0,0,6,1,51,-1,-1
3,201506E098777,1,20,0,0,0,0,1,1,50,4462,1
4,201506E098780,1,9,0,0,0,0,6,1,27,1598,-1


In [18]:
veh2 = mergeOnVars(veh)
veh2.head()

Vehicle_Type
Skidding_and_Overturning
Hit_Object_in_Carriageway
Vehicle_Leaving_Carriageway
Hit_Object_off_Carriageway
Journey_Purpose_of_Driver
Sex_of_Driver


,Accident_Index,Vehicle_Reference,Age_of_Driver,Engine_Capacity_(CC),Age_of_Vehicle,Vehicle_Type,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,Hit_Object_off_Carriageway,Journey_Purpose_of_Driver,Sex_of_Driver
0,201506E098757,2,45,1794,11,Car,None,None,Did not leave carriageway,None,Not known,Male
1,201506E098766,2,51,-1,-1,Car,None,None,Did not leave carriageway,None,Not known,Male
2,201506E098780,1,27,1598,-1,Car,None,None,Did not leave carriageway,None,Not known,Male
3,201506E098804,1,24,1398,1,Car,None,None,Did not leave carriageway,None,Not known,Male
4,201506E098826,1,25,1686,8,Car,None,None,Did not leave carriageway,None,Not known,Male


Export all data to csv.

In [19]:
cas2.to_csv("castualties2015_V.csv")
acc2.to_csv("accidents2015_V.csv")
veh2.to_csv("vehicles2015_V.csv")